## BILSTM_with_attention_Practice

영화 리뷰 데이터를 가지고 긍부정 감정분석을 해보자!

training_data: 15만개

test_data : 5만개

In [129]:
import os
from konlpy.tag import Twitter ## twitter 패키지를 이용하여 토크나이징
import gensim 
import tensorflow as tf
import numpy as np
import codecs
import collections ## word_list 만들때 사용 -> word_embedding은 llok up table 사용할 예정
from rnn_model.attention_bilstm import bilstm_with_attention
from tensorflow.contrib import rnn

In [14]:
def read_data(filename):    
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]        
        data = data[1:]   # header 제외 #    
    return data 
    
train_data = read_data('movie_data/ratings_train.txt') 
test_data = read_data('movie_data/ratings_test.txt') 

In [15]:
train_data ## 리뷰와 긍부정 라벨이 0과 1로 이루어짐

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1'],
 ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'],
 ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'],
 ['9443947',
  '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'],
 ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'],
 ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1'],
 ['9008700', '걍인피니트가짱이다.진짜짱이다♥', '1'],
 ['10217543', '볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~', '1'],
 ['5957425', '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해', '0'],
 ['8628627', '담백하고 깔끔해서 좋다. 신문기사로만 보다 보면 자꾸 잊어버린다. 그들도 사람이었다는 것을.', '1'],
 ['9864035', '취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지', '0'],
 ['6852435', 'ㄱ냥 매번 긴장되고 재밋음ㅠㅠ', '1'],
 ['9143163',
  '참 사람들 웃긴게 

In [16]:
pos_tagger = Twitter() ## 형태소 분석에 twitter package이용

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [17]:
def tokenize(doc):  ## 리스트형테의 데이터 안에서 리뷰부분을 토크나이징해서 다시 리스트로 저장

    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [18]:
train_tokens = [tokenize(row[1]) for row in train_data]
train_labels = [int(row[2]) for row in train_data]
test_tokens = [tokenize(row[1]) for row in test_data]
test_labels = [int(row[2]) for row in test_data]

In [19]:
train_tokens  ## 토큰과 형태소가 같이 저장됨

[['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증나다/Adjective',
  '목소리/Noun'],
 ['흠/Noun',
  '.../Punctuation',
  '포스터/Noun',
  '보고/Noun',
  '초딩/Noun',
  '영화/Noun',
  '줄/Noun',
  '..../Punctuation',
  '오버/Noun',
  '연기/Noun',
  '조차/Josa',
  '가볍다/Adjective',
  '않다/Verb'],
 ['너/Modifier',
  '무재/Noun',
  '밓었/Noun',
  '다그/Noun',
  '래서/Noun',
  '보다/Verb',
  '추천/Noun',
  '한/Josa',
  '다/Adverb'],
 ['교도소/Noun',
  '이야기/Noun',
  '구먼/Noun',
  '../Punctuation',
  '솔직하다/Adjective',
  '재미/Noun',
  '는/Josa',
  '없다/Adjective',
  '../Punctuation',
  '평점/Noun',
  '조정/Noun'],
 ['사이/Modifier',
  '몬페/Noun',
  '그/Determiner',
  '의/Noun',
  '익살스럽다/Adjective',
  '연기/Noun',
  '가/Josa',
  '돋보이다/Verb',
  '영화/Noun',
  '!/Punctuation',
  '스파이더맨/Noun',
  '에서/Josa',
  '늙다/Verb',
  '보이다/Verb',
  '하다/Verb',
  '커스틴/Noun',
  '던스트/Noun',
  '가/Josa',
  '너무나도/Adverb',
  '이쁘다/Adjective',
  '보이다/Verb'],
 ['막/Noun',
  '걸음/Noun',
  '마/Noun',
  '떼다/Verb',
  '3/Number',
  '세/Noun',
  '부터/Josa',
  '초등학교/Noun',

In [20]:
total_tokens = train_tokens + test_tokens

In [21]:
## unlist
from itertools import chain
unlist_tokens = list(chain.from_iterable(total_tokens))

In [22]:
len(total_tokens)

200000

In [23]:
words = list() ## word index를 주기 위해서 dictionary를 만든다 -> lookup table사용
for word in unlist_tokens:
    words.append(word)
word_counter = collections.Counter(words).most_common()
word_dict = dict()
word_dict["<pad>"] = 0
word_dict["<unk>"] = 1
word_dict["<eos>"] = 2
for word,_ in word_counter:
    word_dict[word] = len(word_dict)

In [24]:
word_dict

{'<pad>': 0,
 '<unk>': 1,
 '<eos>': 2,
 './Punctuation': 3,
 '영화/Noun': 4,
 '하다/Verb': 5,
 '이/Josa': 6,
 '보다/Verb': 7,
 '의/Josa': 8,
 '../Punctuation': 9,
 '가/Josa': 10,
 '에/Josa': 11,
 '을/Josa': 12,
 '.../Punctuation': 13,
 '도/Josa': 14,
 '은/Josa': 15,
 '들/Suffix': 16,
 ',/Punctuation': 17,
 '는/Josa': 18,
 '를/Josa': 19,
 '없다/Adjective': 20,
 '있다/Adjective': 21,
 '좋다/Adjective': 22,
 '너무/Adverb': 23,
 '?/Punctuation': 24,
 '이/Determiner': 25,
 '재밌다/Adjective': 26,
 '정말/Noun': 27,
 '것/Noun': 28,
 '되다/Verb': 29,
 '진짜/Noun': 30,
 '!/Punctuation': 31,
 '같다/Adjective': 32,
 '적/Suffix': 33,
 '이/Noun': 34,
 '으로/Josa': 35,
 '점/Noun': 36,
 '아니다/Adjective': 37,
 '않다/Verb': 38,
 '로/Josa': 39,
 '에서/Josa': 40,
 '만/Josa': 41,
 '만들다/Verb': 42,
 '연기/Noun': 43,
 '나오다/Verb': 44,
 '평점/Noun': 45,
 '과/Josa': 46,
 '~/Punctuation': 47,
 '최고/Noun': 48,
 '한/Josa': 49,
 '이다/Verb': 50,
 '왜/Noun': 51,
 '생각/Noun': 52,
 '스토리/Noun': 53,
 '인/Josa': 54,
 '..../Punctuation': 55,
 '다/Adverb': 56,
 '드라마/Noun': 57,
 '다/Jo

In [25]:
vocab_size = len(word_dict)

In [26]:
x = list(map(lambda d: list(map(lambda w: word_dict.get(w, word_dict["<unk>"]), d)), train_tokens))
x = list(map(lambda d: d[:50], x))
x = list(map(lambda d: d + (50- len(d)) * [word_dict["<pad>"]], x))
y = train_labels
train_x = x
train_y = y

In [27]:
x = list(map(lambda d: list(map(lambda w: word_dict.get(w, word_dict["<unk>"]), d)), test_tokens)) ## 딕셔너리의 인덱스들로 바꿔준다.
x = list(map(lambda d: d[:50], x))  ## 최대길이 50까지만 가져옴.
x = list(map(lambda d: d + (50- len(d)) * [word_dict["<pad>"]], x)) ## 50 전에는 padding을 준다.
y = test_labels
test_x = x
test_y = y

In [28]:
def batch_iter(inputs, outputs, batch_size, num_epochs):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(num_epochs):
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, len(inputs))
            yield inputs[start_index:end_index], outputs[start_index:end_index]

In [31]:
NUM_CLASS = 2 ## 긍정: 1 부정: 0
BATCH_SIZE = 64
NUM_EPOCHS = 1    ## epoch 1 로 주고 한번 돌려보자
WORD_MAX_LEN = 50 ## 최대토큰길이 50

In [32]:
num_batches_per_epoch = (len(train_x) - 1) // BATCH_SIZE + 1
print(num_batches_per_epoch)

2344


In [40]:
with tf.Session() as sess:
    
    model = bilstm_with_attention(vocab_size, 50, 2)  ## vocab_size, max_length, class

    init = tf.global_variables_initializer()
    sess.run(init) 




    saver = tf.train.Saver(tf.global_variables())

    train_batches = batch_iter(train_x, train_y, BATCH_SIZE, NUM_EPOCHS)
    num_batches_per_epoch = (len(train_x) - 1) // BATCH_SIZE + 1  ## 2344
    max_accuracy = 0 

    for x_batch, y_batch in train_batches:
        train_feed_dict = {
            model.x: x_batch,
            model.y: y_batch,
            model.is_training: True
        }

        _, step, loss = sess.run([model.optimizer, model.global_step, model.loss], feed_dict=train_feed_dict)

        if step % 100 == 0:
            print("step {0}: loss = {1}".format(step, loss))

        if (step % 1000 == 0) or (step == num_batches_per_epoch ) : 
            # Test accuracy with validation data for each epoch.
            valid_batches = batch_iter(test_x, test_y, BATCH_SIZE, 1) ## test 단에서는 num_epoch 1로 준다
            sum_accuracy, cnt = 0, 0

            for valid_x_batch, valid_y_batch in valid_batches:
                valid_feed_dict = {
                    model.x: valid_x_batch,
                    model.y: valid_y_batch,
                    model.is_training: False
                }

                accuracy = sess.run(model.accuracy, feed_dict=valid_feed_dict)
                sum_accuracy += accuracy
                cnt += 1
            valid_accuracy = sum_accuracy / cnt

            print("\nValidation Accuracy = {1}\n".format(step // num_batches_per_epoch, sum_accuracy / cnt))

            # Save model
            if valid_accuracy > max_accuracy:
                max_accuracy = valid_accuracy
                saver.save(sess, "{0}/{1}.ckpt".format("Movie_attention", "BILSTM_attention"), global_step=step)
                print("Model is saved.\n")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
step 100: loss = 0.6847332715988159
step 200: loss = 0.7001042366027832
step 300: loss = 0.6901065707206726
step 400: loss = 0.6845085620880127
step 500: loss = 0.45540571212768555
step 600: loss = 0.4114309549331665
step 700: loss = 0.43354934453964233
step 800: loss = 0.40674176812171936
step 900: loss = 0.30778220295906067
step 1000: loss = 0.3555498719215393

Validation Accuracy = 0.8370963874680307

Model is saved.

step 1100: loss = 0.2533406913280487
step 1200: loss = 0.23883166909217834
step 1300: loss = 0.41205570101737976
step 1400: loss = 0.307778000831604
step 1500: loss = 0.28070878982543945
step 1600: loss = 0.2867790162563324
step 1700: loss = 0.3154631555080414
step 1800: loss = 0.4692929983139038
step 1900: loss = 0.33120203018188477
step 20

## Test with model

In [125]:
num_class=2
embedding_size = 256
num_hidden = 256
num_layers = 2
learning_rate = 1e-3
tf.reset_default_graph()

x = tf.placeholder(tf.int32, [None, 50], name="x")
x_len = tf.reduce_sum(tf.sign(x), 1)
y = tf.placeholder(tf.int32, [None], name="y")
is_training = tf.placeholder(tf.bool, [], name="is_training")
global_step = tf.Variable(0, trainable=False)
keep_prob = tf.where(is_training, 0.5, 1.0)

with tf.variable_scope("embedding",reuse=tf.AUTO_REUSE):
    init_embeddings = tf.random_uniform([vocab_size, embedding_size])
    embeddings = tf.get_variable("embeddings", initializer=init_embeddings)
    x_emb = tf.nn.embedding_lookup(embeddings, x)

with tf.variable_scope("birnn",reuse=tf.AUTO_REUSE):
    fw_cells = [rnn.BasicLSTMCell(num_hidden) for _ in range(num_layers)]
    bw_cells = [rnn.BasicLSTMCell(num_hidden) for _ in range(num_layers)]
    fw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=keep_prob) for cell in fw_cells]
    bw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=keep_prob) for cell in bw_cells]

    rnn_outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(
    fw_cells, bw_cells, x_emb, sequence_length=x_len, dtype=tf.float32)

with tf.variable_scope("attention",reuse=tf.AUTO_REUSE):
    attention_score = tf.nn.softmax(tf.layers.dense(rnn_outputs, 1, activation=tf.nn.tanh), axis=1)
    attention_out = tf.squeeze(
         tf.matmul(tf.transpose(rnn_outputs, perm=[0, 2, 1]), attention_score),axis=-1)


with tf.variable_scope("output",reuse=tf.AUTO_REUSE):
    logits = tf.layers.dense(attention_out, num_class, activation=tf.nn.softmax)
    predictions = tf.argmax(logits, -1, output_type=tf.int32)
    
with tf.variable_scope("loss",reuse=tf.AUTO_REUSE):
    loss = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

with tf.variable_scope("accuracy",reuse=tf.AUTO_REUSE):
    correct_predictions = tf.equal(predictions, y)
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

saver = tf.train.Saver()
init = tf.global_variables_initializer()
modelName = "Movie_attention/BILSTM_attention.ckpt-2000"

sess = tf.Session()
sess.run(init)
saver.restore(sess, modelName)

probabilities = sess.run(tf.nn.top_k(logits,2,sorted=True), feed_dict = {x: np.array(test_x[3]).reshape(1,50),is_training:False})
print("부정: {}%".format(probabilities[0][0][0]*100),"긍정: {}%".format(probabilities[0][0][1]*100))

attention_weights = sess.run(attention_score,feed_dict = {x: np.array(test_x[3]).reshape(1,50),is_training:False})

INFO:tensorflow:Restoring parameters from BILSTM_attention/BILSTM_attention.ckpt-2000
부정: 98.80242943763733% 긍정: 1.1975673027336597%


In [127]:
test_data[3]

['6825595', '지루하지는 않은데 완전 막장임... 돈주고 보기에는....', '0']

In [126]:
test_tokens[3]

['지루하다/Adjective',
 '않다/Verb',
 '완전/Noun',
 '막장/Noun',
 '임/Noun',
 '.../Punctuation',
 '돈/Noun',
 '주다/Verb',
 '보기/Noun',
 '에는/Josa',
 '..../Punctuation']

## attention score가 높은 상위3개 토큰 추출

In [106]:
attention = [a[0] for a in attention_weights[0]]
enumerated_attention = list(enumerate(attention))
index_sorted_attention= sorted(enumerated_attention, key=lambda x:x[1],reverse=True)
top3_index = [x[0] for x in index_sorted_attention[:3]]

In [124]:
for i in range(3):
    print(test_tokens[3][top3_index[i]])

돈/Noun
막장/Noun
지루하다/Adjective
